In [ ]:
!pip install -U "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git@fc178b5204515ab96dcbced61ea97dc9b9231ab1 "
!pip install --no-deps "xformers<0.0.26" "trl<0.9.0" "peft<0.12.0" "accelerate<0.32.0" "bitsandbytes<0.44.0" "transformers<4.43.0"


  Cloning https://github.com/unslothai/unsloth.git (to revision fc178b5204515ab96dcbced61ea97dc9b9231ab1) to /tmp/pip-install-lwv8s_h8/unsloth_9da92c34e3e34125ac5ebd3f9e6e43ba
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-lwv8s_h8/unsloth_9da92c34e3e34125ac5ebd3f9e6e43ba
  Running command git rev-parse -q --verify 'sha^fc178b5204515ab96dcbced61ea97dc9b9231ab1'
  Running command git fetch -q https://github.com/unslothai/unsloth.git fc178b5204515ab96dcbced61ea97dc9b9231ab1
  Running command git checkout -q fc178b5204515ab96dcbced61ea97dc9b9231ab1
  Resolved https://github.com/unslothai/unsloth.git to commit fc178b5204515ab96dcbced61ea97dc9b9231ab1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.5 MB

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.10: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# Load the full training dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Shuffle the dataset for randomness and create our smaller splits
shuffled_dataset = full_dataset.shuffle(seed=42)
train_dataset = shuffled_dataset.select(range(8000))
validation_dataset = shuffled_dataset.select(range(8000, 8200))

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# The instructional prompt template for training
training_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Carefully analyze each step of the provided solution and determine if it is correct. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Reasoning and Verification:
{}
Output:
{}"""

# We must add an End Of Sequence (EOS) token to tell the model when a completion is finished.
EOS_TOKEN = tokenizer.eos_token

# This function formats our data samples into the prompt template.
def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]
    texts = []
    for question, solution, output in zip(questions, solutions, outputs):
        # Format the prompt and add the EOS token
        reasoning_placeholder = ""
        text = training_prompt.format(question, str(solution), reasoning_placeholder, str(output)) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

# Apply the formatting function to our training dataset
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True, load_from_cache_file=False,)
formatted_val_dataset = validation_dataset.map(formatting_prompts_func, batched=True, load_from_cache_file=False,)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64,
    lora_dropout = 0.1,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.10 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    eval_dataset = formatted_val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 6)],
    args = TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        num_train_epochs=5,
        learning_rate=1e-4,
        weight_decay=0.01,
        logging_steps=10,
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=200,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        warmup_ratio = 0.05,
        lr_scheduler_type="cosine_with_restarts",
        prediction_loss_only=True,
        include_inputs_for_metrics=False,
        eval_accumulation_steps=4,
        per_device_eval_batch_size=4,
        dataloader_pin_memory=False,
        dataloader_drop_last=True,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        optim = "adamw_8bit",
        seed = 42,
        output_dir = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpointv2",
        save_total_limit=2,
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/8000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,000 | Num Epochs = 5 | Total steps = 2,500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,0.684200,0.651755
200,0.643800,0.634204
300,0.663800,0.626065
400,0.639100,0.616716
500,0.634800,0.612404
600,0.529900,0.616413
700,0.548900,0.606942
800,0.516400,0.606023
900,0.524100,0.599219
1000,0.549400,0.591439


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=1600, training_loss=0.5276304449141026, metrics={'train_runtime': 7679.5399, 'train_samples_per_second': 5.209, 'train_steps_per_second': 0.326, 'total_flos': 4.4189089507948954e+17, 'train_loss': 0.5276304449141026, 'epoch': 3.2})

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/gdrive/MyDrive/llama3_8b_math_verifier_checkpointv2"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Model checkpoint and tokenizer saved to: /content/gdrive/MyDrive/llama3_8b_math_verifier_checkpointv2


In [ ]:
# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Carefully analyze each step of the provided solution and determine if it is correct. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Reasoning and Verification:
Output:
"""

# Select a sample from the validation set
example = validation_dataset[10] # You can change the index (e.g., to 1, 2, 50)
question = example["question"]
solution = example["solution"]

# Format the prompt with the validation data
inference_text = inference_prompt.format(question, solution)
inputs = tokenizer([inference_text], return_tensors="pt").to("cuda")

# Generate the model's response
outputs = model.generate(**inputs,max_new_tokens=64,do_sample=False,temperature=0,use_cache=True)
response = tokenizer.batch_decode(outputs)

# Print the results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
# We process the output to show only the generated text
print(response[0].split("Output:\n")[1])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

#### QUESTION ####
Lily has $3$ gallons of milk and gives $\frac{12}{5}$ gallons to James. What fraction of a gallon does she have left?

#### SOLUTION ####
Let's use Python's sympy library to solve this arithmetic problem.
<llm-code>
from sympy import Rational

# Define the amount of milk Lily has and the amount she gives to James
lily_milk = 3
james_milk = Rational(12,5)

# Calculate the fraction of milk Lily has left
lily_left = lily_milk - james_milk

print(lily_left)
</llm-code>
<llm-code-output>
3/5
</llm-code-output>
So Lily has $\boxed{\frac{3}{5}}$ of a gallon left.

#### MODEL'S PREDICTION ####
True<|end_of_text|>

#### CORRECT ANSWER ####
True


In [ ]:
import pandas as pd
from tqdm import tqdm
from google.colab import files
# Load the official test set

test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=64,temperature=0,do_sample=False,use_cache=True)
    response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)
submission_path = "/content/submission.csv"
files.download(submission_path)
print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

 11%|█         | 1064/10000 [04:44<40:00,  3.72it/s]Unsloth: Input IDs of shape torch.Size([1, 1050]) with length 1050 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 13%|█▎        | 1274/10000 [05:40<39:13,  3.71it/s]Unsloth: Input IDs of shape torch.Size([1, 1218]) with length 1218 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 16%|█▌        | 1572/10000 [07:05<37:33,  3.74it/s]Unsloth: Input IDs of shape torch.Size([1, 1059]) with length 1059 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 16%|█▌        | 1590/10000 [07:10<37:23,  3.75it/s]Unsloth: Input IDs of shape torch.Size([1, 1031]) with length 1031 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 23%|██▎       | 2329/10000 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.
